<a href="https://colab.research.google.com/github/khoinguyen-hvkn/MaSSP/blob/master/RNN/rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Recurrent Neural Networks - Generative Language

In [0]:
# Import các thư viện cần thiết
import csv
import itertools
import operator
import numpy as np
# Sử dụng thư viện nltk (Natural Language Toolkit) để phân tách dữ liệu thô
import nltk
import sys
from datetime import datetime

import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
def softmax(x):
    xt = np.exp(x - np.max(x))
    return xt / np.sum(xt)

In [0]:
# Tải dữ liệu cho NLTK toolkit
nltk.download("book")

### Mô hình hóa ngôn ngữ
Mục tiêu của ta là xây dựng một mô hình ngôn ngữ sử dụng RNN. Giả sử ta có một câu với $m$ từ, thì một mô hình ngôn ngữ cho phép ta dự đoán được xác xuất của một câu (trong tập dữ liệu) là:
$
\begin{aligned}
P(w_1,...,w_m) = \prod_{i=1}^{m}P(w_i \mid w_1,..., w_{i-1}) 
\end{aligned}
$
Có thể hiểu, xác suất của mỗi câu chính là tích xác suất của các từ với điều kiện đã biết các từ xuất hiện phía trước nó. Ví dụ xác suất của câu "Mentee yêu các mentor" sẽ bằng xác suất của "mentor" khi đã biết các từ "Mentee yêu các" nhân với xác suất của "các" khi đã biết "Mentee yêu", ...

Ưu điểm của phương pháp này là gì ? Và tại sao cần sử dụng nó? Nó có thể được sử dụng làm metrics đánh giá. Ví dụ một machine translations có khả năng tạo ra được nhiều câu dịch, tuy nhiên nó sẽ lựa chọn câu có xác suất lớn nhất. Cách đánh giá này tương tự hệ thống nhận dạng giọng nói.

Và vì ta có thể tính được xác suất của một từ khi biết các từ đã xuất hiện trước đó, nên ta có thể làm hệ thống tự sinh văn bản (một dạng của mô hình sinh). Ta lấy một vài từ của một cầu rồi chọn dần các từ còn lại với xác suất dự đoán tốt nhất  cho tới khi ta có một câu hoàn thiện. Cứ lặp lại các bước như vậy ta sẽ có một văn bản tự sinh.

Lưu ý rằng công thức xác xuất ở trên của mỗi từ là xác xuất có điều kiện khi biết trước tất cả các từ trước nó. Trong thực tế, bởi khả năng tính toán và bộ nhớ của máy tính có hạn, nên với nhiều mô hình ta khó có thể biểu diễn được những phụ thuộc xa (long-term dependences). Vì vậy mà ta chỉ xem được một vài từ trước đó. Về mặt lý thuyết, RNN có thể xử lý được cả các phụ thuộc xa của các câu dài, nhưng trên thực tế nó lại khá phức tạp, và gặp phải các vấn đề như vanishing. LSTM là một mở rộng của RNN nhằm giải quyết vấn đề này.

### Tiền xử lý dữ liệu
Để huấn luyện mô hình ngôn ngữ, ta cần dữ liệu là văn bản để làm dữ liệu huấn luyện. Dữ liệu 15,000 bình luận reddit được tải từ cơ sở dữ liệu [BigQuery của Google](https://bigquery.cloud.google.com/table/fh-bigquery:reddit_comments.2015_08). Tác giả lưu trữ dữ liệu ở file *reddit-comments-2015-08.csv*.

####1. Tokenize Text
Chúng ta có dữ liệu thô, và mục đích là dự đoán từng từ, do đó chúng ta cần phân tách dữ liệu thành các từ riêng biệt, bao gồm cả các dấu câu. Ví dụ "Tôi yêu MaSSP!" cần chia thành 4 phần: "Tôi", "yêu", "MaSSP" và "!". Để thuận tiện, ta sẽ sử dụng NLTK với 2 hàm chính *word_tokenize* và *sent_tokenize* để phân tách dữ liệu.

#### 2. Loại bỏ các từ ít gặp

Trong hầu hết các văn bản có những từ ta rất hiếm khi thấy nó xuất hiện, những từ này ta hoàn toàn có thể loại bỏ. Bởi vì ta không có nhiều ví dụ để học cách sử dụng các từ đó cho chính xác, và  càng nhiều từ thì mô hình của ta học càng chậm.

Ta giới hạn lượng từ vựng phổ biến bằng biến *vocabulary_size*. Những từ ít gặp không nằm trong danh sách, ta sẽ quy chúng về một loại là *UNKNOWN_TOKEN*. Ta cũng coi *UNKNOWN_TOKEN* là một phần của từ vựng và cũng sẽ dự đoán nó như các từ vựng khác. Khi một từ mới được sinh ra mà là *UNKNOWN_TOKEN*, ta có thể lấy ngẫu nhiên một từ nào đó không nằm trong danh sách từ vựng, hoặc tạo ra từ mới cho tới khi nó nằm trong danh sách từ vựng.

#### 3. Thêm kí tự đầu, cuối
Ta thêm vào 2 kí tự đặc biệt cho mỗi câu là `SENTENCE_START` và `SENTENCE_END` biểu thị cho từ bắt đầu và từ kết thúc của câu. Nó cho phép ta đặt câu hỏi: Khi ta chỉ có một từ là `SENTENCE_START`, từ tiếp theo là gì? Câu trả lời chính là từ đầu tiên của câu.

#### 4. Encode dữ liệu
Đầu vào của RNN là các vector dữ liệu chứa số thứ tự của các từ trong từ điển. Ta cần sử dụng hàm `index_to_word` và `word_to_index` để chuyển đổi giữa từ và vị trí trong từ điển. Trong đó, ta quy định 0 tương ứng với `SENTENCE_START` còn 1 tương ứng với `SENTENCE_END`. Ví dụ $x$ có dạng `[0, 69, 96, 6996, 111]`
Vì mục tiêu của ta là dự đoán các từ tiếp theo nên  $y$ sẽ là dịch một ví trí so với $x$, và kết thúc câu là `SENTENCE_END`. Vậy dự đoán chính xác nhất sẽ là `[69, 96, 6996, 111, 1]`.



In [0]:
vocabulary_size = 8000
unknown_token = "UNKNOWN_TOKEN"
sentence_start_token = "SENTENCE_START"
sentence_end_token = "SENTENCE_END"

# Đọc dữ liệu và thêm token SENTENCE_START và SENTENCE_END
print "Reading CSV file..."
with open('reddit-comments-2015-08.csv', 'rb') as f:
    reader = csv.reader(f, skipinitialspace=True)
    reader.next()
    # Phân tách các comments sử dụng nltk
    sentences = itertools.chain(*[nltk.sent_tokenize(x[0].decode('utf-8').lower()) for x in reader])
    # Thêm token SENTENCE_START Và SENTENCE_END
    sentences = ["%s %s %s" % (sentence_start_token, x, sentence_end_token) for x in sentences]
print "Parsed %d sentences." % (len(sentences))
    
# Phân tách câu thành các từ
tokenized_sentences = [nltk.word_tokenize(sent) for sent in sentences]

# Đếm tần suất xuất hiện của từ
word_freq = nltk.FreqDist(itertools.chain(*tokenized_sentences))
print "Found %d unique words tokens." % len(word_freq.items())

# Tìm ra các từ phổ biến nhất, xây dựng bộ từ điển
vocab = word_freq.most_common(vocabulary_size-1)
index_to_word = [x[0] for x in vocab]
index_to_word.append(unknown_token)
word_to_index = dict([(w,i) for i,w in enumerate(index_to_word)])

print "Using vocabulary size %d." % vocabulary_size
print "The least frequent word in our vocabulary is '%s' and appeared %d times." % (vocab[-1][0], vocab[-1][1])

# Thay thế các từ không nằm trong từ điển bởi `unknown token`, lưu kết quả tiền xử lý câu
for i, sent in enumerate(tokenized_sentences):
    tokenized_sentences[i] = [w if w in word_to_index else unknown_token for w in sent]

print "\nExample sentence: '%s'" % sentences[0]
print "\nExample sentence after Pre-processing: '%s'" % tokenized_sentences[0]

Reading CSV file...
Parsed 79170 sentences.
Found 65498 unique words tokens.
Using vocabulary size 8000.
The least frequent word in our vocabulary is 'traction' and appeared 10 times.

Example sentence: 'SENTENCE_START i joined a new league this year and they have different scoring rules than i'm used to. SENTENCE_END'

Example sentence after Pre-processing: '[u'SENTENCE_START', u'i', u'joined', u'a', u'new', u'league', u'this', u'year', u'and', u'they', u'have', u'different', u'scoring', u'rules', u'than', u'i', u"'m", u'used', u'to', u'.', u'SENTENCE_END']'


In [0]:
# Khởi tạo dữ liệu training
X_train = np.asarray([[word_to_index[w] for w in sent[:-1]] for sent in tokenized_sentences])
y_train = np.asarray([[word_to_index[w] for w in sent[1:]] for sent in tokenized_sentences])

Một ví dụ về quá trình tiền xử lý dữ liệu từ một câu đơn:

In [0]:
# Print an training data example
x_example, y_example = X_train[17], y_train[17]
print "x:\n%s\n%s" % (" ".join([index_to_word[x] for x in x_example]), x_example)
print "\ny:\n%s\n%s" % (" ".join([index_to_word[x] for x in y_example]), y_example)

x:
SENTENCE_START what are n't you understanding about this ? !
[0, 51, 27, 16, 10, 861, 54, 25, 34, 69]

y:
what are n't you understanding about this ? ! SENTENCE_END
[51, 27, 16, 10, 861, 54, 25, 34, 69, 1]


#### Xây dựng RNN

![](http://www.wildml.com/wp-content/uploads/2015/09/rnn.jpg)

Quan sát mô hình mạng trên. Đầu vào $x$ là chuỗi các từ đầu vào và $x_t$ là từ ở bước thứ $t$. Có một điều đáng chú ý: Bởi vì phép nhân ma trận không cho phép sử dụng địa chỉ của từ để làm việc, do đó ta phải sử dụng `one-hot vector` với kích cỡ bằng kích cỡ từ điển `vocabulary_size`. Do đó, mỗi $x_t$ sẽ là một vector, và $x$ là một ma trận, vỡi mỗi hàng biểu diễn cho một từ. Chúng ta thực hiện transform này trong phần triển khai Neural Network thay vì trong phần tiền xử lý. Kết quả của mạng $o$ cũng có kích cỡ tương tự. Mỗi $o_t$ là một vector của phần tử trong từ điển, kích cỡ `vocabulary_size`, và mỗi phần tử đại diện cho xác suất từ tương ứng là từ tiếp theo trong câu.

Viết lại công thức của mạng RNN:
$
\begin{aligned}
s_t &= \tanh(Ux_t + Ws_{t-1}) \\
o_t &= \mathrm{softmax}(Vs_t)
\end{aligned}
$

Giả sử chúng ta sử dụng từ điển với kích cỡ $C=8000$ và một lớp ẩn kích cỡ $H = 100$ (Bộ nhớ của mạng). Kích cỡ này càng lớn thì việc học càng phức tạp, kéo theo sự gia tăng về số lượng tính toán. Ta có chiều của các dữ liệu như sau:

$
\begin{aligned}
x_t & \in \mathbb{R}^{8000} \\
o_t & \in \mathbb{R}^{8000} \\
s_t & \in \mathbb{R}^{100} \\
U & \in \mathbb{R}^{100 \times 8000} \\
V & \in \mathbb{R}^{8000 \times 100} \\
W & \in \mathbb{R}^{100 \times 100} \\
\end{aligned}
$


$U,V$ và $W$ là tham số của mạng chúng ta muốn học từ dữ liệu. Do đó, ta cần phải học tất cả $2HC + H^2$ tham số. Các tham số này cho thấy nút thắt của mô hình khi hoạt động. Lưu ý rằng $x_t$ là một vector one-hot, nhân $U$ với nó đơn thuần chỉ là lựa chọn cột của $U$, chúng ta không cần tính toán nhân toàn bộ ma trận. Do đó trong các công thức trên, phép tính toán lớn nhất là phép tính $V s_t$. Đó là lý do tại sao chúng ta muốn giữ số lượng từ vựng nhỏ nhất có thể.

#### Khởi tạo
Chúng ta bắt đầu mạng RNN bởi việc khởi tạo các tham số. Trong bước này chúng ta tạo ra class `RNNNumpy`. Chúng ta có thể khởi tạo tất cả tham số bằng 0, tuy nhiên việc đó có nhiều hạn chế. Chúng ta có thể khởi tạo nó ngẫu nhiên. Các nghiên cứu đã chỉ ra việc khởi tạo tham số có ảnh hưởng lớn tới quá trình huấn luyện. Và việc khởi tạo còn phụ thuộc vào activation function của ta. Trong trường hợp activation function là hàm `tanh` như ở trên, giá trị khởi tạo thường được khởi tạo trong $\left[-\frac{1}{\sqrt{n}}, \frac{1}{\sqrt{n}}\right]$ trong đó $n$ là số lượng kết nối đến từ layer trước. Và chúng ta khởi tạo tham số ngẫu nhiên đủ nhỏ thì mạng sẽ hoạt động tốt.

In [0]:
class RNNNumpy:
    
    def __init__(self, word_dim, hidden_dim=100, bptt_truncate=4):
        # Assign instance variables
        self.word_dim = word_dim
        self.hidden_dim = hidden_dim
        self.bptt_truncate = bptt_truncate
        # Randomly initialize the network parameters
        self.U = np.random.uniform(-np.sqrt(1./word_dim), np.sqrt(1./word_dim), (hidden_dim, word_dim))
        self.V = np.random.uniform(-np.sqrt(1./hidden_dim), np.sqrt(1./hidden_dim), (word_dim, hidden_dim))
        self.W = np.random.uniform(-np.sqrt(1./hidden_dim), np.sqrt(1./hidden_dim), (hidden_dim, hidden_dim))
        

`word_dim` là kích cỡ của từ điển, và `hidden_size` là kích cỡ của lớp ẩn. `bptt_truncate` là các tham số cho quá trình tính đạo hàm.

#### Forward Propagation


In [0]:
def forward_propagation(self, x):
    # Số bước thời gian
    T = len(x)
    # Trong suốt quá trình propagation chúng ta lữu trữ toàn bộ trạng thái ẩn trong s
    # Ta thêm vào một hàng cho lớp ẩn, set bằng 0
    s = np.zeros((T + 1, self.hidden_dim))
    s[-1] = np.zeros(self.hidden_dim)
    # Kết quả đầu ra tại mỗi bước thời gian. Chúng ta cũng lưu lại phục vụ tính toán sau này
    o = np.zeros((T, self.word_dim))
    # Với mỗi bước thời gian
    for t in np.arange(T):
        # U. x[t] đơn giản là lựa chọn cột x[t] của U. Chính là việc nhân U với một one-hot vector.
        s[t] = np.tanh(self.U[:,x[t]] + self.W.dot(s[t-1]))
        o[t] = softmax(self.V.dot(s[t]))
    return [o, s]

RNNNumpy.forward_propagation = forward_propagation

Ta không chỉ tính toán outputs, mà còn cho các trạng thái ẩn. Ta sử dụng ở phía sau để tính toán đạo hàm. Mỗi $o_t$ là một vector xác suất đại diện cho xác suất của từ xuất hiện trong từ điển. Ta thường sử dụng từ có xác suất cao nhất, ta gọi hàm này là `predict`.

In [0]:
def predict(self, x):
    # Thực hiện forward propagation và trả về phần tử có xác suất cao nhất
    o, s = self.forward_propagation(x)
    return np.argmax(o, axis=1)

RNNNumpy.predict = predict

Thực hiện ví dụ sau

In [0]:
np.random.seed(10)
model = RNNNumpy(vocabulary_size)
o, s = model.forward_propagation(X_train[10])
print o.shape
print o

(45, 8000)
[[0.00012408 0.0001244  0.00012603 ... 0.00012515 0.00012488 0.00012508]
 [0.00012536 0.00012582 0.00012436 ... 0.00012482 0.00012456 0.00012451]
 [0.00012387 0.0001252  0.00012474 ... 0.00012559 0.00012588 0.00012551]
 ...
 [0.00012406 0.00012463 0.00012539 ... 0.00012617 0.00012463 0.00012589]
 [0.00012547 0.00012431 0.00012485 ... 0.00012427 0.00012611 0.00012472]
 [0.00012482 0.00012529 0.00012477 ... 0.00012488 0.00012508 0.0001267 ]]


Với mỗi từ trong câu sau (45 time step), mô hình tạo ra 8000 dự đoán cho xác suất của từ tiếp theo. Ta khởi tạo $U, V, W$ ngẫu nhiên.

In [0]:
predictions = model.predict(X_train[10])
print predictions.shape
print predictions

(45,)
[1284 5221 7653 7430 1013 3562 7366 1627 7290 3251 7299 6722  565  238
 2539   21 6548  261 5274 2082 1835 5376 3522  477 7051 7352 7715 3822
 6914 5059 3850 6176  743 2082 5561 2182 6569 2800 2752 6821 4437 7021
 6399 6912 3922]


#### Tính toán hàm mất mát
Để huấn luyện mạng ta sẽ sử dụng hàm cross-entropy. Với $N$ là số lượng mẫu huấn luyện và $C$ là số class (kích cỡ của từ điển) ta có hàm mất mát tương tứng với dự đoán $o$ và kết quả đúng $y$:
$
\begin{aligned}
L(y,o) = - \frac{1}{N} \sum_{n \in N} y_{n} \log o_{n}
\end{aligned}
$

In [0]:
def calculate_total_loss(self, x, y):
    L = 0
    # For each sentence...
    for i in np.arange(len(y)):
        o, s = self.forward_propagation(x[i])
        # We only care about our prediction of the "correct" words
        correct_word_predictions = o[np.arange(len(y[i])), y[i]]
        # Add to the loss based on how off we were
        L += -1 * np.sum(np.log(correct_word_predictions))
    return L

def calculate_loss(self, x, y):
    # Divide the total loss by the number of training examples
    N = np.sum((len(y_i) for y_i in y))
    return self.calculate_total_loss(x,y)/N

RNNNumpy.calculate_total_loss = calculate_total_loss
RNNNumpy.calculate_loss = calculate_loss

Let's take a step back and think about what the loss should be for random predictions. That will give us a baseline and make sure our implementation is correct. We have $C$ words in our vocabulary, so each word should be (on average) predicted with probability $1/C$, which would yield a loss of $L = -\frac{1}{N} N \log\frac{1}{C} = \log C$:

In [0]:
# Limit to 1000 examples to save time
print "Expected Loss for random predictions: %f" % np.log(vocabulary_size)
print "Actual loss: %f" % model.calculate_loss(X_train[:1000], y_train[:1000])

Expected Loss for random predictions: 8.987197


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  


Actual loss: 8.987430


Pretty close! Keep in mind that evaluating the loss on the full dataset is an expensive operation and can take hours if you have a lot of data!

#### Training the RNN with SGD and Backpropagation Through Time (BPTT)

Remember that we want to find the parameters $U,V$ and $W$ that minimize the total loss on the training data. The most common way to do this is SGD, Stochastic Gradient Descent. The idea behind SGD is pretty simple. We iterate over all our training examples and during each iteration we nudge the parameters into a direction that reduces the error. These directions are given by the gradients on the loss: $\frac{\partial L}{\partial U}, \frac{\partial L}{\partial V}, \frac{\partial L}{\partial W}$. SGD also needs a *learning rate*, which defines how big of a step we want to make in each iteration. SGD is the most popular optimization method not only for Neural Networks, but also for many other Machine Learning algorithms. As such there has been a lot of research on how to optimize SGD using batching, parallelism and adaptive learning rates. Even though the basic idea is simple, implementing SGD in a really efficient way can become very complex. If you want to learn more about SGD [this](http://cs231n.github.io/optimization-1/) is a good place to start. Due to its popularity there are a wealth of tutorials floating around the web, and I don't want to duplicate them here. I'll implement a simple version of SGD that should be understandable even without a background in optimization.

But how do we calculate those gradients we mentioned above? In a [traditional Neural Network](http://www.wildml.com/2015/09/implementing-a-neural-network-from-scratch/) we do this through the backpropagation algorithm. In RNNs we use a slightly modified version of the this algorithm called Backpropagation Through Time (BPTT). Because the parameters are shared by all time steps in the network, the gradient at each output depends not only on the calculations of the current time step, but also the previous time steps. If you know calculus, it really is just applying the chain rule. The next part of the tutorial will be all about BPTT, so I won't go into detailed derivation here. For a general introduction to backpropagation check out [this](http://colah.github.io/posts/2015-08-Backprop/) and this [post](http://cs231n.github.io/optimization-2/). For now you can treat BPTT as a black box. It takes as input a training example $(x,y)$ and returns the gradients $\frac{\partial L}{\partial U}, \frac{\partial L}{\partial V}, \frac{\partial L}{\partial W}$.

In [0]:
def bptt(self, x, y):
    T = len(y)
    # Perform forward propagation
    o, s = self.forward_propagation(x)
    # We accumulate the gradients in these variables
    dLdU = np.zeros(self.U.shape)
    dLdV = np.zeros(self.V.shape)
    dLdW = np.zeros(self.W.shape)
    delta_o = o
    delta_o[np.arange(len(y)), y] -= 1.
    # For each output backwards...
    for t in np.arange(T)[::-1]:
        dLdV += np.outer(delta_o[t], s[t].T)
        # Initial delta calculation
        delta_t = self.V.T.dot(delta_o[t]) * (1 - (s[t] ** 2))
        # Backpropagation through time (for at most self.bptt_truncate steps)
        for bptt_step in np.arange(max(0, t-self.bptt_truncate), t+1)[::-1]:
            # print "Backpropagation step t=%d bptt step=%d " % (t, bptt_step)
            dLdW += np.outer(delta_t, s[bptt_step-1])              
            dLdU[:,x[bptt_step]] += delta_t
            # Update delta for next step
            delta_t = self.W.T.dot(delta_t) * (1 - s[bptt_step-1] ** 2)
    return [dLdU, dLdV, dLdW]

RNNNumpy.bptt = bptt

#### Gradient Checking

Whenever you implement backpropagation it is good idea to also implement *gradient checking*, which is a way of verifying that your implementation is correct. The idea behind gradient checking is that derivative of a parameter is equal to the slope at the point, which we can approximate by slightly changing the parameter and then dividing by the change:

$
\begin{aligned}
\frac{\partial L}{\partial \theta} \approx \lim_{h \to 0} \frac{J(\theta + h) - J(\theta -h)}{2h}
\end{aligned}
$

We then compare the gradient we calculated using backpropagation to the gradient we estimated with the method above. If there's no large difference we are good. The approximation needs to calculate the total loss for *every* parameter, so that gradient checking is very expensive (remember, we had more than a million parameters in the example above). So it's a good idea to perform it on a model with a smaller vocabulary.

In [0]:
def gradient_check(self, x, y, h=0.001, error_threshold=0.01):
    # Calculate the gradients using backpropagation. We want to checker if these are correct.
    bptt_gradients = model.bptt(x, y)
    # List of all parameters we want to check.
    model_parameters = ['U', 'V', 'W']
    # Gradient check for each parameter
    for pidx, pname in enumerate(model_parameters):
        # Get the actual parameter value from the mode, e.g. model.W
        parameter = operator.attrgetter(pname)(self)
        print "Performing gradient check for parameter %s with size %d." % (pname, np.prod(parameter.shape))
        # Iterate over each element of the parameter matrix, e.g. (0,0), (0,1), ...
        it = np.nditer(parameter, flags=['multi_index'], op_flags=['readwrite'])
        while not it.finished:
            ix = it.multi_index
            # Save the original value so we can reset it later
            original_value = parameter[ix]
            # Estimate the gradient using (f(x+h) - f(x-h))/(2*h)
            parameter[ix] = original_value + h
            gradplus = model.calculate_total_loss([x],[y])
            parameter[ix] = original_value - h
            gradminus = model.calculate_total_loss([x],[y])
            estimated_gradient = (gradplus - gradminus)/(2*h)
            # Reset parameter to original value
            parameter[ix] = original_value
            # The gradient for this parameter calculated using backpropagation
            backprop_gradient = bptt_gradients[pidx][ix]
            # calculate The relative error: (|x - y|/(|x| + |y|))
            relative_error = np.abs(backprop_gradient - estimated_gradient)/(np.abs(backprop_gradient) + np.abs(estimated_gradient))
            # If the error is to large fail the gradient check
            if relative_error > error_threshold:
                print "Gradient Check ERROR: parameter=%s ix=%s" % (pname, ix)
                print "+h Loss: %f" % gradplus
                print "-h Loss: %f" % gradminus
                print "Estimated_gradient: %f" % estimated_gradient
                print "Backpropagation gradient: %f" % backprop_gradient
                print "Relative Error: %f" % relative_error
                return 
            it.iternext()
        print "Gradient check for parameter %s passed." % (pname)

RNNNumpy.gradient_check = gradient_check

# To avoid performing millions of expensive calculations we use a smaller vocabulary size for checking.
grad_check_vocab_size = 100
np.random.seed(10)
model = RNNNumpy(grad_check_vocab_size, 10, bptt_truncate=1000)
model.gradient_check([0,1,2,3], [1,2,3,4])

Performing gradient check for parameter U with size 1000.


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:28: RuntimeWarning: invalid value encountered in double_scalars


Gradient check for parameter U passed.
Performing gradient check for parameter V with size 1000.
Gradient check for parameter V passed.
Performing gradient check for parameter W with size 100.
Gradient check for parameter W passed.


#### SGD Implementation

Now that we are able to calculate the gradients for our parameters we can implement SGD. I like to do this in two steps: 1. A function `sdg_step` that calculates the gradients and performs the updates for one batch. 2. An outer loop that iterates through the training set and adjusts the learning rate.

In [0]:
# Performs one step of SGD.
def numpy_sdg_step(self, x, y, learning_rate):
    # Calculate the gradients
    dLdU, dLdV, dLdW = self.bptt(x, y)
    # Change parameters according to gradients and learning rate
    self.U -= learning_rate * dLdU
    self.V -= learning_rate * dLdV
    self.W -= learning_rate * dLdW

RNNNumpy.sgd_step = numpy_sdg_step

In [0]:
# Outer SGD Loop
# - model: The RNN model instance
# - X_train: The training data set
# - y_train: The training data labels
# - learning_rate: Initial learning rate for SGD
# - nepoch: Number of times to iterate through the complete dataset
# - evaluate_loss_after: Evaluate the loss after this many epochs
def train_with_sgd(model, X_train, y_train, learning_rate=0.005, nepoch=100, evaluate_loss_after=5):
    # We keep track of the losses so we can plot them later
    losses = []
    num_examples_seen = 0
    for epoch in range(nepoch):
        # Optionally evaluate the loss
        if (epoch % evaluate_loss_after == 0):
            loss = model.calculate_loss(X_train, y_train)
            losses.append((num_examples_seen, loss))
            time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            print "%s: Loss after num_examples_seen=%d epoch=%d: %f" % (time, num_examples_seen, epoch, loss)
            # Adjust the learning rate if loss increases
            if (len(losses) > 1 and losses[-1][1] > losses[-2][1]):
                learning_rate = learning_rate * 0.5  
                print "Setting learning rate to %f" % learning_rate
            sys.stdout.flush()
        # For each training example...
        for i in range(len(y_train)):
            # One SGD step
            model.sgd_step(X_train[i], y_train[i], learning_rate)
            num_examples_seen += 1

Done! Let's try to get a sense of how long it would take to train our network:

In [0]:
np.random.seed(10)
model = RNNNumpy(vocabulary_size)
%timeit model.sgd_step(X_train[10], y_train[10], 0.005)

1 loop, best of 3: 266 ms per loop


Uh-oh, bad news. One step of SGD takes approximately 350 milliseconds on my laptop. We have about 80,000 examples in our training data, so one epoch (iteration over the whole data set) would take several hours. Multiple epochs would take days, or even weeks! And we're still working with a small dataset compared to what's being used by many of the companies and researchers out there. What now?

Fortunately there are many ways to speed up our code. We could stick with the same model and make our code run faster, or we could modify our model to be less computationally expensive, or both. Researchers have identified many ways to make models less computationally expensive, for example by using a hierarchical softmax or adding projection layers to avoid the large matrix multiplications (see also [here](http://arxiv.org/pdf/1301.3781.pdf) or [here](http://www.fit.vutbr.cz/research/groups/speech/publi/2011/mikolov_icassp2011_5528.pdf)). But I want to keep our model simple and go the first route: Make our implementation run faster using a GPU. Before doing that though, let's just try to run SGD with a small dataset and check if the loss actually decreases:

In [0]:
np.random.seed(10)
# Train on a small subset of the data to see what happens
model = RNNNumpy(vocabulary_size)
losses = train_with_sgd(model, X_train[:1000], y_train[:1000], nepoch=20, evaluate_loss_after=1)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  


2019-07-02 16:11:20: Loss after num_examples_seen=0 epoch=0: 8.987430
2019-07-02 16:13:46: Loss after num_examples_seen=1000 epoch=1: 6.097781
2019-07-02 16:16:10: Loss after num_examples_seen=2000 epoch=2: 5.887186
2019-07-02 16:18:34: Loss after num_examples_seen=3000 epoch=3: 5.787054
2019-07-02 16:20:59: Loss after num_examples_seen=4000 epoch=4: 5.706788
2019-07-02 16:23:25: Loss after num_examples_seen=5000 epoch=5: 5.633586
2019-07-02 16:25:48: Loss after num_examples_seen=6000 epoch=6: 5.554554
2019-07-02 16:28:14: Loss after num_examples_seen=7000 epoch=7: 5.471431
2019-07-02 16:30:40: Loss after num_examples_seen=8000 epoch=8: 5.405841
2019-07-02 16:33:05: Loss after num_examples_seen=9000 epoch=9: 5.366256
2019-07-02 16:35:26: Loss after num_examples_seen=10000 epoch=10: 5.323700
2019-07-02 16:37:47: Loss after num_examples_seen=11000 epoch=11: 5.276175
2019-07-02 16:40:10: Loss after num_examples_seen=12000 epoch=12: 5.252626
2019-07-02 16:42:33: Loss after num_examples_see

Good, it seems like our implementation is at least doing something useful and decreasing the loss, just like we wanted.

### Generating Text 

Now that we have our model we can ask it to generate new text for us! Let's implement a helper function to generate new sentences:

In [0]:
def generate_sentence(model):
    # We start the sentence with the start token
    new_sentence = [word_to_index[sentence_start_token]]
    # Repeat until we get an end token
    while not new_sentence[-1] == word_to_index[sentence_end_token]:
        next_word_probs = model.forward_propagation(new_sentence)[0]
        sampled_word = word_to_index[unknown_token]
        # We don't want to sample unknown words
        while sampled_word == word_to_index[unknown_token]:
            #print(next_word_probs[-1])
            samples = np.random.multinomial(1, next_word_probs[-1])
            sampled_word = np.argmax(samples)
        new_sentence.append(sampled_word)
    sentence_str = [index_to_word[x] for x in new_sentence[1:-1]]
    return sentence_str

num_sentences = 30
senten_min_length = 10

for i in range(num_sentences):
    sent = []
    # We want long sentences, not sentences with one or two words
    while len(sent) < senten_min_length:
        sent = generate_sentence(model)
    print " ".join(sent)

the clothing does . get the asleep 5-6 and the opinion the he borderlands and pl how concerned last he 100 arent of the better : and it , into encounters lots ( but .
http : anyone stupid bad thing working with other with *if ] ( https : take . .
** detailed the cups of very receiving with your hope and be common that and do n't think the everyone .
as people codes | the usd less informed and so areas kind , so figured by to be different sucked does n't have them who gay cash youre me fucking sudden this on ^ conservative that would the audio immediately .
worst is just really how pc place mega all out prison .
] ( still : a trump some sound google .
that does tell else the scoring from ll to your provide useful and true about the same and do see .
acceptable the do to playing a automatically or early an quality .
threw threaten like your bot 140 you have some had as either .
shit some for a credibility team but would the going even mod of that is all of best to all very themselves her

In [0]:
print(X_train.shape)

(79170,)
